In [1]:
from langchain.schema  import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv

c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [3]:
docs = [
    Document(
        page_content="Peak Performance Gym was founded in 2015 by former Olympic athlete Marcus Chen. With over 15 years of experience in professional athletics, Marcus established the gym to provide personalized fitness solutions for people of all levels. The gym spans 10,000 square feet and features state-of-the-art equipment.",
        metadata={"source": "about.txt"}
    ),
    Document(
        page_content="Peak Performance Gym is open Monday through Friday from 5:00 AM to 11:00 PM. On weekends, our hours are 7:00 AM to 9:00 PM. We remain closed on major national holidays. Members with Premium access can enter using their key cards 24/7, including holidays.",
        metadata={"source": "hours.txt"}
    ),
    Document(
        page_content="Our membership plans include: Basic (₹1,500/month) with access to gym floor and basic equipment; Standard (₹2,500/month) adds group classes and locker facilities; Premium (₹4,000/month) includes 24/7 access, personal training sessions, and spa facilities. We offer student and senior citizen discounts of 15% on all plans. Corporate partnerships are available for companies with 10+ employees joining.",
        metadata={"source": "membership.txt"}
    ),
    Document(
        page_content="Group fitness classes at Peak Performance Gym include Yoga (beginner, intermediate, advanced), HIIT, Zumba, Spin Cycling, CrossFit, and Pilates. Beginner classes are held every Monday and Wednesday at 6:00 PM. Intermediate and advanced classes are scheduled throughout the week. The full schedule is available on our mobile app or at the reception desk.",
        metadata={"source": "classes.txt"}
    ),
    Document(
        page_content="Personal trainers at Peak Performance Gym are all certified professionals with minimum 5 years of experience. Each new member receives a complimentary fitness assessment and one free session with a trainer. Our head trainer, Neha Kapoor, specializes in rehabilitation fitness and sports-specific training. Personal training sessions can be booked individually (₹800/session) or in packages of 10 (₹7,000) or 20 (₹13,000).",
        metadata={"source": "trainers.txt"}
    ),
    Document(
        page_content="Peak Performance Gym's facilities include a cardio zone with 30+ machines, strength training area, functional fitness space, dedicated yoga studio, spin class room, swimming pool (25m), sauna and steam rooms, juice bar, and locker rooms with shower facilities. Our equipment is replaced or upgraded every 3 years to ensure members have access to the latest fitness technology.",
        metadata={"source": "facilities.txt"}
    )
]

db = Chroma.from_documents(docs, embedding_function)

In [4]:
retriver = db.as_retriever(search_type="mmr", search_kwargs={"k":3})

In [5]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import tool

retriver_tool = create_retriever_tool(
    retriever=retriver,
    name="retriever_tool",
    description="Information related to Gym History & Founder, Operating Hours, Membership Plans, Fitness Classes, Personal Trainers, and Facilities & Equipment of Peak Performance Gym",
)

@tool
def off_topic():
    """Catch all Questions NOT related to Peak Performance Gym's history, hours, membership plans, fitness classes, trainers, or facilities"""
    return "Forbidden - do not respond to the user"

tools = [retriver_tool, off_topic]

In [6]:
from typing import Annotated,Sequence, Literal, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages : Annotated[ Sequence[BaseMessage], add_messages]

In [11]:

from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import END, StateGraph, START


llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash")
def agent(state: AgentState) :
    messages = state["messages"]
    model = llm.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state: AgentState) -> Literal["tools", END]:
    latest_message = state["messages"][-1]
    if latest_message.tool_calls :
        return "tools"
    return END


In [12]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)
workflow= StateGraph(AgentState)
workflow.add_node("agent",agent)
workflow.add_node("tools",tool_node)
workflow.add_edge(START,"agent")
workflow.add_conditional_edges("agent",should_continue)

workflow.add_edge("tools","agent")
graph = workflow.compile()

In [24]:
# from IPython.display import Image, display
# from langchain_core.runnables.graph import MermaidDrawMethod

# display(
#     Image(
#         graph.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API,
#         )
#     )
# )

In [21]:
graph.invoke(
    input={"messages": [HumanMessage(content="How will the weather be tommorrow?")]}
)

{'messages': [HumanMessage(content='How will the weather be tommorrow?', additional_kwargs={}, response_metadata={}, id='47453009-afc4-4a05-a6ea-8680db15745c'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'off_topic', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--dc3fc907-851a-4315-88d6-a006fc9200aa-0', tool_calls=[{'name': 'off_topic', 'args': {}, 'id': '8ff24c6c-79dc-4e10-8bec-3f89be216ac9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 121, 'output_tokens': 10, 'total_tokens': 185, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='Forbidden - do not respond to the user', name='off_topic', id='1a0e842a-c3e4-4e29-ba57-256c5827f081', tool_call_id='8ff24c6c-79dc-4e10-8bec-3f89be216ac9'),
  AIMessage(content='I cannot provide weather forecasts. My purpose is to provide information about Peak Performance Gym.', additional_kwarg

In [22]:
graph.invoke(input={
    "messages": [HumanMessage(content="Who is the owner and what are the timings?")]
})

{'messages': [HumanMessage(content='Who is the owner and what are the timings?', additional_kwargs={}, response_metadata={}, id='7ac99b03-9e99-4562-bf27-67a0c182e827'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'retriever_tool', 'arguments': '{"query": "What are the operating hours of Peak Performance Gym?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--17d0e53a-f48d-4ed0-992d-0e05b651e53a-0', tool_calls=[{'name': 'retriever_tool', 'args': {'query': 'Who is the owner of Peak Performance Gym?'}, 'id': '4086c122-0358-4a5e-8311-25fe6734a008', 'type': 'tool_call'}, {'name': 'retriever_tool', 'args': {'query': 'What are the operating hours of Peak Performance Gym?'}, 'id': 'cd096eb5-40c5-415c-ad1c-b1a190e3590e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 49, 'total_tokens': 276, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(